In [1]:
%run ../ipynb_util_tars.py

In [2]:
import torch
from transformers import AutoTokenizer
from lxt.models.llama import LlamaForCausalLM, attnlrp
from lxt.utils import pdf_heatmap, clean_tokens

model = LlamaForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# apply AttnLRP rules
attnlrp.register(model)

prompt = """\
Context: Mount Everest attracts many climbers, including highly experienced mountaineers. There are two main climbing routes, one approaching the summit from the southeast in Nepal (known as the standard route) and the other from the north in Tibet. While not posing substantial technical climbing challenges on the standard route, Everest presents dangers such as altitude sickness, weather, and wind, as well as hazards from avalanches and the Khumbu Icefall. As of November 2022, 310 people have died on Everest. Over 200 bodies remain on the mountain and have not been removed due to the dangerous conditions. The first recorded efforts to reach Everest's summit were made by British mountaineers. As Nepal did not allow foreigners to enter the country at the time, the British made several attempts on the north ridge route from the Tibetan side. After the first reconnaissance expedition by the British in 1921 reached 7,000 m (22,970 ft) on the North Col, the 1922 expedition pushed the north ridge route up to 8,320 m (27,300 ft), marking the first time a human had climbed above 8,000 m (26,247 ft). The 1924 expedition resulted in one of the greatest mysteries on Everest to this day: George Mallory and Andrew Irvine made a final summit attempt on 8 June but never returned, sparking debate as to whether they were the first to reach the top. Tenzing Norgay and Edmund Hillary made the first documented ascent of Everest in 1953, using the southeast ridge route. Norgay had reached 8,595 m (28,199 ft) the previous year as a member of the 1952 Swiss expedition. The Chinese mountaineering team of Wang Fuzhou, Gonpo, and Qu Yinhua made the first reported ascent of the peak from the north ridge on 25 May 1960. \
Question: How high did they climb in 1922? According to the text, the 1922 expedition reached 8,"""

input_ids = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).input_ids.to(model.device)
input_embeds = model.get_input_embeddings()(input_ids)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [3]:
output_logits = model(inputs_embeds=input_embeds.requires_grad_(), use_cache=False).logits

In [4]:
output_logits.shape

torch.Size([1, 504, 32000])

In [5]:
max_logits, max_indices = torch.max(output_logits[0, -1, :], dim=-1)

max_logits.backward(max_logits)
relevance = input_embeds.grad.float().sum(-1).cpu()[0]

# normalize relevance between [-1, 1] for plotting
relevance = relevance / relevance.abs().max()

# remove '_' characters from token strings
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
tokens = clean_tokens(tokens)

In [6]:
relevance

tensor([ 1.4142e-01,  4.9700e-03,  3.2170e-03,  3.7260e-03,  3.6056e-03,
         4.6697e-03,  1.8035e-03,  1.2307e-03,  1.6535e-03,  2.9311e-03,
         2.4982e-03, -1.1494e-03,  1.7481e-03,  1.9444e-03,  1.7738e-03,
         1.9680e-03,  2.4556e-03,  2.2394e-03,  1.8183e-03,  1.9807e-03,
         1.9668e-03,  1.9497e-03,  1.4409e-03,  2.9973e-03,  2.1020e-03,
         2.2702e-03,  1.1746e-03,  4.6093e-04,  2.0909e-03,  1.4651e-03,
         2.4700e-03,  1.1895e-03,  1.8469e-03,  1.2871e-03,  1.7337e-03,
         1.6185e-03,  2.3092e-03,  1.8725e-03,  1.8802e-03,  1.2827e-03,
         5.5377e-04,  1.2248e-03,  1.0094e-03,  1.2231e-03,  1.0380e-03,
         1.6856e-03,  1.5208e-03,  1.2405e-03,  5.1707e-04,  1.2809e-03,
         9.2748e-04,  1.2127e-03,  1.8236e-03,  1.8319e-03,  1.6120e-03,
         3.2192e-03,  7.5262e-04,  1.2913e-03,  1.1210e-03,  1.2961e-03,
         1.2042e-03,  1.8866e-03,  3.0032e-03,  2.1106e-03,  1.3845e-03,
         1.3297e-03,  1.6040e-03,  1.0562e-03,  1.1

In [8]:
pdf_heatmap(tokens, relevance, path='heatmap.pdf', backend='xelatex')

FileNotFoundError: [Errno 2] No such file or directory: 'xelatex'